In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pandas as pd
import matplotlib.pyplot as plt

import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkFiles

import sklearn as skl
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords, wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
46 packages can be upgraded. Run 'apt list --upgradable' to see them.


In [24]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
from pyspark.sql import SparkSession

#spark = SparkSession.builder.appName("CreateDataFrame").getOrCreate()
spark = SparkSession.builder \
    .appName("Read CSV into DataFrame") \
    .getOrCreate()

In [3]:
file_path = '/content/drive/MyDrive/Course Relevance Dataset.csv'

# Read CSV into DataFrame
#course_data_df = spark.read.csv(file_path, header=True, inferSchema=True)
spark.sparkContext.addFile(file_path)
course_data_df = spark.read\
.option("multiline", True)\
.csv(SparkFiles.get("Course Relevance Dataset.csv"), sep=",", header=True)
course_data_df.show(10)


+----+-------------------+--------------------+---------+--------------------+-----------------------+
|SrNo|Name Of the Program|      Type of Course|     Code|                Need|Description of the need|
+----+-------------------+--------------------+---------+--------------------+-----------------------+
|   1|     BA (Geography)|Environmental stu...| UGEVS101|Local, Regional, ...|   To make student u...|
|   2|     BA (Geography)|Environmental stu...| UGEVS201|Local, Regional, ...|   Examining histori...|
|   3|     BA (Geography)|Environmental Sci...|  UGESlOl|Local, Regional, ...|   To make student u...|
|   4|     BA (Geography)|Environmental Sci...|  UGES201|Local, Regional, ...|   Managing waste is...|
|   5|     BA (Geography)|Fundamentals of G...| UGGEO101|Local, Regional, ...|   These geological ...|
|   6|     BA (Geography)|     Human Geography| UGGEO201|Local, Regional, ...|   Cultural factors ...|
|   7|     BA (Geography)|Physical Geograph...| UGGEO302|            Nati

In [ ]:
#pd.set_option('max_colwidth', None)

In [4]:
course_data_pandas = course_data_df.toPandas()
course_data_pandas.head(10)

,SrNo,Name Of the Program,Type of Course,Code,Need,Description of the need
0,1,BA (Geography),Environmental studies,UGEVS101,"Local, Regional, National and Global","To make student understand the distribution, u..."
1,2,BA (Geography),Environmental studies,UGEVS201,"Local, Regional, National and Global",Examining historical and current environmental...
2,3,BA (Geography),Environmental Science-I,UGESlOl,"Local, Regional, National and Global","To make student understand the distribution, u..."
3,4,BA (Geography),Environmental Science-II,UGES201,"Local, Regional, National and Global",Managing waste is critical for preventing envi...
4,5,BA (Geography),Fundamentals of Geomorphology,UGGEO101,"Local, Regional, National and Global",These geological processes shape the Earth's s...
5,6,BA (Geography),Human Geography,UGGEO201,"Local, Regional, National and Global",Cultural factors influence human-environment i...
6,7,BA (Geography),Physical Geography of India,UGGEO302,National,Understanding drainage patterns and climate is...
7,8,BA (Geography),Agricultural Geography of India,UGGEO402,National,"In the context of Indian agriculture, these to..."
8,9,BA (Geography),Geography of Settlements,UGGEO 501,"Local, Regional, National and Global",studying settlement patterns helps in understa...
9,10,BA (Geography),Tools and Techniques in Geography for SpatialA...,UGGEO 503,"Local, Regional, National and Global",Toposheets aid in map reading and interpretati...


In [5]:
nltk.download('udhr')

[nltk_data] Downloading package udhr to /root/nltk_data...
[nltk_data]   Package udhr is already up-to-date!


True

In [ ]:
#nltk.stem.snowball.demo()

In [6]:
row_nan_count = course_data_pandas.isnull().sum()
print(row_nan_count)

SrNo                       0
Name Of the Program        1
Type of Course             1
Code                       1
Need                       2
Description of the need    1
dtype: int64


In [7]:
#  NLTK data - Only need to run once (I think).
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
# sample = course_data_pandas['Description of the need'][1]

In [ ]:
## Source: https://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python

# from collections import defaultdict

# lemmatizer = WordNetLemmatizer()

# tokens = word_tokenize(sample.lower())

# tag_map = defaultdict(lambda : wn.NOUN)
# tag_map['J'] = wn.ADJ
# tag_map['V'] = wn.VERB
# tag_map['R'] = wn.ADV

# l = []
# for token, tag in pos_tag(tokens):
#     lemma = lemmatizer.lemmatize(token, tag_map[tag[0]])
#     l.append(lemma)
# print(l)

In [9]:
course_data_pandas['Robust'] = course_data_pandas['Name Of the Program'] + ' ' + course_data_pandas['Type of Course'] + ' ' + \
course_data_pandas['Description of the need']

course_data_pandas.head()

,SrNo,Name Of the Program,Type of Course,Code,Need,Description of the need,Robust
0,1,BA (Geography),Environmental studies,UGEVS101,"Local, Regional, National and Global","To make student understand the distribution, u...",BA (Geography) Environmental studies To make s...
1,2,BA (Geography),Environmental studies,UGEVS201,"Local, Regional, National and Global",Examining historical and current environmental...,BA (Geography) Environmental studies Examining...
2,3,BA (Geography),Environmental Science-I,UGESlOl,"Local, Regional, National and Global","To make student understand the distribution, u...",BA (Geography) Environmental Science-I To make...
3,4,BA (Geography),Environmental Science-II,UGES201,"Local, Regional, National and Global",Managing waste is critical for preventing envi...,BA (Geography) Environmental Science-II Managi...
4,5,BA (Geography),Fundamentals of Geomorphology,UGGEO101,"Local, Regional, National and Global",These geological processes shape the Earth's s...,BA (Geography) Fundamentals of Geomorphology T...


In [23]:
row_nan_count = course_data_pandas.isnull().sum()
print(row_nan_count)

SrNo                       0
Name Of the Program        1
Type of Course             1
Code                       1
Need                       2
Description of the need    1
Robust                     0
lemmatized_description     0
dtype: int64


In [16]:
# https://www.nltk.org/_modules/nltk/stem/wordnet.html
lemmatizer = WordNetLemmatizer()

# Function that processes the text input
def preprocess_text(text):
  # Handle potential float values
    if isinstance(text, float):
      text = str(text)
    #Tokenize text
    tokens = word_tokenize(text.lower())
    # Remove stopwords and numbers
    tokens = [word for word in tokens if word.isalpha() and word not in stopwords.words('english')]
    # Lemmatize tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # print(tokens)
    return ' '.join(tokens)


# Run function and add to processed column
course_data_pandas['lemmatized_description'] = course_data_pandas['Robust'].apply(preprocess_text)

course_data_pandas[['Robust', 'lemmatized_description']].head()

,Robust,lemmatized_description
0,BA (Geography) Environmental studies To make s...,ba geography environmental study make student ...
1,BA (Geography) Environmental studies Examining...,ba geography environmental study examining his...
2,BA (Geography) Environmental Science-I To make...,ba geography environmental make student unders...
3,BA (Geography) Environmental Science-II Managi...,ba geography environmental managing waste crit...
4,BA (Geography) Fundamentals of Geomorphology T...,ba geography fundamental geomorphology geologi...


In [ ]:
# # https://www.nltk.org/_modules/nltk/stem/wordnet.html
# lemmatizer = WordNetLemmatizer()

# # Process text
# def preprocess_text(text):
#     if text is None:
#               return ""
#     #Tokenize text
#     tokens = word_tokenize(text.lower())
#     # Remove stopwords and numbers
#     tokens = [word for word in tokens if word.isalpha() and word not in stopwords.words('english')]
#     # Lemmatize tokens
#     tokens = [lemmatizer.lemmatize(word) for word in tokens]
#     # print(tokens)
#     return ' '.join(tokens)

# # Run function and add to processed column
# course_data_pandas['processed_description'] = course_data_pandas['Description of the need'].apply(preprocess_text)
# course_data_pandas[['Description of the need', 'processed_description']].head()

In [17]:
print(lemmatizer.lemmatize('universal'))

universal


In [18]:
course_data_pandas.head()

,SrNo,Name Of the Program,Type of Course,Code,Need,Description of the need,Robust,lemmatized_description
0,1,BA (Geography),Environmental studies,UGEVS101,"Local, Regional, National and Global","To make student understand the distribution, u...",BA (Geography) Environmental studies To make s...,ba geography environmental study make student ...
1,2,BA (Geography),Environmental studies,UGEVS201,"Local, Regional, National and Global",Examining historical and current environmental...,BA (Geography) Environmental studies Examining...,ba geography environmental study examining his...
2,3,BA (Geography),Environmental Science-I,UGESlOl,"Local, Regional, National and Global","To make student understand the distribution, u...",BA (Geography) Environmental Science-I To make...,ba geography environmental make student unders...
3,4,BA (Geography),Environmental Science-II,UGES201,"Local, Regional, National and Global",Managing waste is critical for preventing envi...,BA (Geography) Environmental Science-II Managi...,ba geography environmental managing waste crit...
4,5,BA (Geography),Fundamentals of Geomorphology,UGGEO101,"Local, Regional, National and Global",These geological processes shape the Earth's s...,BA (Geography) Fundamentals of Geomorphology T...,ba geography fundamental geomorphology geologi...


In [24]:
#course_data_pandas['Robust'] = course_data_pandas['Robust'].astype(str)
course_data_pandas['Name Of the Program'] = course_data_pandas['Name Of the Program'].astype(str)
course_data_pandas['Type of Course'] = course_data_pandas['Type of Course'].astype(str)
course_data_pandas['Code'] = course_data_pandas['Code'].astype(str)
course_data_pandas['Need'] = course_data_pandas['Need'].astype(str)
course_data_pandas['Description of the need'] = course_data_pandas['Description of the need'].astype(str)

In [22]:
corpus = course_data_pandas['Robust']

tfidf_vectorizer = TfidfVectorizer()
count_vectorizer = CountVectorizer()

x = tfidf_vectorizer.fit_transform(corpus)
tfidf_vectorizer.get_feature_names_out()

array(['11', '1550', '16th', ..., 'your', 'zealand', 'zoology'],
      dtype=object)

In [25]:
x.shape

(880, 2603)

In [26]:
# Vectorizer

corpus2 = count_vectorizer.fit_transform(corpus)
#print(corpus2)

print(count_vectorizer.get_feature_names_out())

['11' '1550' '16th' ... 'your' 'zealand' 'zoology']


In [27]:
cv_df = pd.DataFrame(corpus2.toarray(), columns=count_vectorizer.get_feature_names_out())

In [28]:
cv_df

,11,1550,16th,1832,18th,1980,19th,1cybersecurity,1potential,1pv4,...,wrapper,write,writing,wsns,years,yield,you,your,zealand,zoology
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
876,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
877,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
878,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(n_neighbors=5, metric='cosine')

model.fit(x)

# This function will find and return recommended courses, n = # of results returned
def course_finder(description, n=5):
    # Use the preprocess function built above
    processed_description = preprocess_text(description)
    # Transform the input description into TF-IDF features - similar to corpus vectorizer above
    description_tfidf = tfidf_vectorizer.transform([processed_description])
    # Find KNN
    distances, indices = model.kneighbors(description_tfidf, n_neighbors=n)
    recommendations = course_data_pandas.iloc[indices[0]]
    return recommendations[['Code', 'Name Of the Program', 'Type of Course', 'Description of the need']]

# Example usage: Recommend courses based on a sample description
course_description = "I want to learn about natural language processing"
course_recommendation = course_finder(course_description)
course_recommendation

,Code,Name Of the Program,Type of Course,Description of the need
842,PGIT403,MSc (Information\nTechnology),Natural Language Processing,Natural language processing helps computers co...
781,PGDS303,MSC (Data Science),Natural Language Processing,NLP can be applied to enhance communication be...
651,PGCHODSE404 A,MSc (Organic Chemistry),Natural Products and Heterocvclic Chemistry,Students are exposed to reaction mechanisms wh...
843,PGIT4P3,MSc (Information Technology),Natural Language Processing Practical,Technologies are critical for enterprises that...
668,PGMB104- DSEC- 2,MSc (Microbiology),Food&Dairy Microbiology,Students will learn various aspects of microbi...
